# Homework Assignment #4 (Individual)
## Regression and SVM Predictive Models

### <p style="text-align: right;"> &#9989; Nick Balesky</p>
### <p style="text-align: right;"> &#9989; NickCoder27</p>

### Goal for this homework assignment
By now, you have learned a bit about predictive models. In this assignment, you will practice:

* Building predictive models using `statsmodels` and `sklearn`
* Evaluating your predictive models

**This assignment is due roughly two weeks from now at 11:59 pm on Friday, April 21st.** It should be uploaded into the "Homework Assignments" submission folder for Homework #4.  Submission instructions can be found at the end of the notebook.


---
## Part 0: Add to your Git repository to track your progress on your assignment (4 points)

For this assignment, you're going to add it to the `cmse202-s23-turnin` repository you created in class so that you can track your progress on the assignment and preserve the final version that you turn in. In order to do this you need to

**&#9989; Do the following**:

1. Navigate to your `cmse202-s23-turnin` repository and create a new directory called `hw-04`.
2. Move this notebook into that **new directory** in your repository, then **add it and commit it to your repository**.
1. Finally, to test that everything is working, "git push" the file so that it ends up in your GitHub repository.

**Important**: Double check you've added your Professor and your TA as collaborators to your "turnin" repository (you should have done this in the previous homework assignment).

**Also important**: Make sure that the version of this notebook that you are working on is the same one that you just added to your repository! If you are working on a different copy of the notebook, **none of your changes will be tracked**!

If everything went as intended, the file should now show up on your GitHub account in the "`cmse202-s23-turnin`" repository inside the `hw-04` directory that you just created.  Periodically, **you'll be asked to commit your changes to the repository and push them to the remote GitHub location**. Of course, you can always commit your changes more often than that, if you wish.  It can be good to get into a habit of committing your changes any time you make a significant modification, or when you stop working on the project for a bit.

&#9989; **Do this**: Before you move on, put the command that your instructor should run to clone your repository in the markdown cell below.

https://github.com/NickCoder27/CMSE202-s23-turnin.git

# Building a Model to Predict if it Will Rain Tomorrow

In this dataset we have weather data for a number of Australian cities. We’re going to **build a model that uses data from today to predict if it will rain tomorrow.** To simplify our model, let’s just look at the weather for one city.

## Part 1. Working with the Data

The dataset that we’ll be using can be found here:
- `https://raw.githubusercontent.com/msu-cmse-courses/cmse202-S23-data/main/HW/HW4/aussie_weather_data.csv`

#### 1.1 (3 Points)
**Download the data set and read it in using Pandas.**

In [35]:
#Write your code here

import pandas as pd

weather_data = pd.read_csv("aussie_weather_data.csv")

weather_data = weather_data.rename(columns={"DayOfMonth": "Day"})

weather_data["Date"] = pd.to_datetime(weather_data[["Year", "Month", "Day"]])

# Now, you can drop the original 'Year', 'Month', and 'Day' columns if you don't need them anymore
weather_data = weather_data.drop(columns=["Year", "Month", "Day"])

weather_data.head()

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Date
0,NorfolkIsland,20.4,25.8,0.0,6.0,12.4,112.5,31.0,112.5,135.0,...,71.0,1014.5,1013.6,3.0,1.0,23.3,24.7,0,0,2009-01-01
1,NorfolkIsland,20.9,26.7,0.2,8.0,10.3,90.0,31.0,45.0,67.5,...,77.0,1016.3,1015.5,2.0,5.0,25.0,25.1,0,0,2009-01-02
2,NorfolkIsland,22.3,26.3,0.0,3.2,2.0,0.0,35.0,22.5,315.0,...,90.0,1014.6,1014.3,7.0,7.0,24.7,23.8,0,1,2009-01-03
3,NorfolkIsland,21.6,22.2,1.2,2.8,0.0,337.5,41.0,337.5,337.5,...,95.0,1016.0,1015.3,8.0,8.0,22.1,21.2,1,1,2009-01-04
4,NorfolkIsland,20.4,23.5,2.6,2.2,2.9,337.5,52.0,337.5,337.5,...,86.0,1015.3,1013.7,7.0,7.0,21.8,21.6,1,0,2009-01-05


#### 1.2 (3 Points)
This dataset has weather information for multiple cities in Australia. It’s quite a large dataset. In fact, it’s a little *too* large for our purposes; it will take a considerable amount of time to train a model on so much data. So we’ll just work with the data for a specific city.

The first thing we’ll do is see which cities we have data for. 
**Print out a list of the specific cities in this dataset.** (Note that this is not the same as printing the entire `Location` column.)

In [36]:
#Write your code here

unique_cities = []

for i in weather_data["Location"]:
    if i not in unique_cities:
        unique_cities.append(i)
    else:
        pass
    
unique_cities

['NorfolkIsland',
 'SydneyAirport',
 'WaggaWagga',
 'MelbourneAirport',
 'Mildura',
 'Watsonia',
 'Brisbane',
 'Cairns',
 'Townsville',
 'MountGambier',
 'Nuriootpa',
 'PerthAirport',
 'Perth',
 'AliceSprings',
 'Darwin']

#### 1.3 (3 Points)
Select one of the cities (and **please** don't just pick the first one!). Cut down your dataframe so that it only contains data from your chosen city. You should also **look through the columns in this dataset and ensure that all of the data is ready for our analysis/model creation.**

In [37]:
#Write your code here

perth_data = weather_data[weather_data["Location"]=="Perth"]

perth_data = perth_data.drop(columns=["Date", "Location"]).dropna()
perth_data


,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
31205,6.4,20.7,0.0,1.8,7.0,67.5,22.0,337.5,22.5,6.0,...,80.0,39.0,1024.1,1019.0,0.0,6.0,11.1,19.7,0,0
31206,9.5,19.2,1.8,1.2,4.7,180.0,26.0,45.0,112.5,11.0,...,93.0,73.0,1019.3,1018.4,6.0,6.0,13.2,17.7,1,1
31207,9.5,16.4,1.8,1.4,4.9,202.5,44.0,180.0,225.0,13.0,...,69.0,57.0,1020.4,1022.1,7.0,5.0,15.9,16.0,1,1
31208,0.7,15.9,6.8,2.4,9.3,45.0,24.0,22.5,67.5,4.0,...,86.0,41.0,1032.0,1029.6,0.0,1.0,6.9,15.5,1,0
31209,0.7,18.3,0.0,0.8,9.3,90.0,37.0,67.5,45.0,15.0,...,72.0,36.0,1028.9,1024.2,1.0,5.0,8.7,17.9,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34225,10.3,19.9,0.2,1.8,7.5,135.0,37.0,45.0,112.5,9.0,...,89.0,60.0,1017.1,1013.8,5.0,6.0,13.0,18.5,0,1
34226,13.0,16.8,61.2,3.6,0.0,247.5,46.0,180.0,225.0,17.0,...,90.0,75.0,1005.6,1008.9,7.0,7.0,16.4,15.6,1,0
34227,13.3,18.9,0.4,1.8,6.5,315.0,37.0,315.0,337.5,11.0,...,85.0,65.0,1019.2,1019.4,6.0,6.0,15.1,18.0,0,0
34228,11.5,18.2,0.0,3.8,9.3,315.0,30.0,337.5,337.5,9.0,...,62.0,47.0,1025.9,1023.4,1.0,3.0,14.0,17.6,0,0


#### 1.4 (3 Points)
Finally, before we start creating our models, let's first split our data into training and testing datasets. **Keep in mind what we want our model to predict and how this dataset gives you the information you need for your model.**

In [38]:
#Write your code here
from sklearn.model_selection import train_test_split

X = perth_data.drop(columns=["RainTomorrow"])
y = perth_data["RainTomorrow"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


## 2.0 Building a Regression Model

#### 2.1 (3 Points)
We’ll start by creating a regression model. **Does it make sense for us to use linear regression or logistic regression? Explain your choice.**

It makes sense to use logistic regression given the binary data of 1 (yes it will rain) and 0 (no it will not rain) while linear regression works best for continuous data

#### 2.2 (12 Points)
Create a regression model. Use the model summary to determine which parameters are most statistically significant in your model. If you get an error during at this stage it may be helpful to go back to part 1.3 and ensure that all of your data is appropriate for creating a model. 

In [41]:
#Write your code here

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Fit the logistic regression model using scikit-learn
log_reg = LogisticRegression(max_iter=5000)
log_reg.fit(X_train, y_train)

# Predict on the test set and calculate the accuracy
y_pred = log_reg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Logistic Regression Model Accuracy: {accuracy:.4f}")

# Show the feature importances (coefficients) for each predictor variable
feature_importances = pd.DataFrame({"Feature": X_train.columns, "Importance": log_reg.coef_[0]})
feature_importances = feature_importances.sort_values(by="Importance", ascending=False)
print(feature_importances)



Logistic Regression Model Accuracy: 0.8976
          Feature  Importance
19      RainToday    0.466866
13    Pressure9am    0.305725
17        Temp9am    0.116096
6   WindGustSpeed    0.106847
1         MaxTemp    0.084317
16       Cloud3pm    0.078412
12    Humidity3pm    0.065604
10   WindSpeed3pm    0.042644
15       Cloud9am    0.033612
2        Rainfall    0.023398
11    Humidity9am    0.011825
5     WindGustDir   -0.001961
7      WindDir9am   -0.002473
8      WindDir3pm   -0.003325
9    WindSpeed9am   -0.031151
18        Temp3pm   -0.040559
3     Evaporation   -0.142604
0         MinTemp   -0.153817
4        Sunshine   -0.250758
14    Pressure3pm   -0.313976


RainToday, Pressure9am, Temp9am, and WindGustSpeed are the most correlated with RainTomorrow and my model will predict with 89.76% accuracy

#### 2.3 (8 Points)
**Fit your test data and create/print a confusion matrix. We’ll use this to evaluate how well your predicitve model performs.**

In [42]:
#Write your code here

from sklearn.metrics import confusion_matrix

# Predict on the test set
y_pred = log_reg.predict(X_test)

# Create the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Print the confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[693  29]
 [ 64 122]]


#### 2.4 (3 Points)
Look at/calculate the Precision and Recall of your model. **Which one is greater? Describe in plain language what that means about the performance of your model (I.e., the circumstances in which it does/doesn’t do well).**

In [43]:
from sklearn.metrics import precision_score, recall_score

# Calculate precision and recall
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

# Print the results
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

Precision: 0.8079
Recall: 0.6559


Precision is greather than recall so my model is more reliable when it predicts rain tomorrow, but may not be good at identifying all rainy days and may miss some (lower recall). 

## 3.0 Interlude: A Base Rate Frequency Model (12 Points)

If we didn't create a fancy model for predicting future rainfall, would that mean that we had *no way* of predicting if it would rain? Of course not. There's always *some* kind of model we can use (they just won't be very sophisticated). 

In the case of our problem--predicting whether it will rain tomorrow--what is the simplest model we could use? We'd need to look at the **base rate frequency**. If we know that, over the course of the year, it rains ~30% of the time, that would be our base rate frequency. If we were to guess that it would be sunny every day of the year, we'd be wrong ~30% of the time, *but we'd be right ~70% of the time.* 

Why is this important? Imagine we create a sophisticated model that accurately predicts rain/no rain ~60% of the time; without any other information, we might think, "That's not too bad, I guess." It certainly feels better than *no* information, right? Now think about the base rate frequency model, which achieves an accuracy of **70%** just by guessing the same thing every time. Suddenly, our sophisticated model seems like crap! It tells us we could be doing ~10% better than our fancy model by guessing. This is why we use the base rate frequency as a simple way to test our models.


**Calculate how frequently it rains in your city. If you were to guess that it *wasn't* going to rain every day, how frequently would you be right? (We'll use this as our baseline model.)**


In [57]:
#Write your code for calculating the frequency of rain here.

rain_frequency = weather_data["RainToday"].sum() / weather_data.shape[0]

rain_frequency

0.21467797039099076

It rains 21.5% of the time in Perth, Australia so if I guess it WASN'T going to rain every day I would be correct 78.5% of the time

---

### &#128721; STOP (3 Point)
**Pause to commit your changes to your Git repository!**

Take a moment to save your notebook, commit the changes to your Git repository with a meaningful commit message.

---

## 4.0 SVM Model

We’ve seen how regression faired in predicting rain; now let’s give Support Vector Machines a shot. 

It’s not clear, *a priori*, which type of kernel (linear or RBF) would fair better. Therefore, we’ll need to test out both kernel types, as well as multiple values for hyperparameters ($C$ and $\gamma$). 

#### 4.1 (12 Points)
Run a grid search over both types of kernels and multiple values of $C$ and $\gamma$ (we recommend [0.01,0.1,1.0,10.0] for both). We *STRONGLY* recommend you set the parameter `n_jobs=-1` in `GridSearchCV`, which will distribute the computational load. Make sure to write down your best-fit hyperparameters. 

In [ ]:
#Write your code here

*Write down your best fir hyper-parameters here.*

#### 4.2 (8 Points)
Using your best-fit parameters, classify your test data and print off the confusion matrix. Note/calculate the Precision and Recall for your SVM model, as well as the overall accuracy.

$\mathrm{Accuracy} = \frac{\mathrm{Accurate~Predictions}}{\mathrm{Total~Samples}}$.

In [ ]:
#Write your code here

## 5. Evaluation of Models

Here we’re going to investigate how well our predictive models actually work.


#### 5.1 (5 Points)

Compare the accuracy of both of your predictive models (regression and SVM) to the baseline model from part 3. Do your predictive models do better than the baseline model? Is their performance relative to the baseline model what *you* would expect? 


*Write down your response here.*

#### 5.2 (5 Points)

Let’s test the robustness of your model. Before you do anything, you should record the precision, recall, and accuracy of your two predictive models (if you haven’t already). Once you’ve done that, go all the way back to part 1.4 and change the random seed for `train_test_split`. Using this new data split, rerun all of your code from parts 2 and 4. For each new random seed, record the precision, recall, and accuracy for both models (and write them, as well as the random seed, in the cell below!). Do this for 3-5 different random seeds. 


*Write down the accuracy, precision, recall, and seed number for each of your iterations.*

#### 5.3 (5 Points)

How much variation did you observe in the precision, recall, and accuracy in your models? After going through this exercise, how (if at all) does this change your response to part 5.1? 

*Write down your response here.*

#### 5.4 (5 Points)

Finally, compare the performance of your regression and SVM models. Which, if any, performed better? Esxplain.

*Write down your response here.*

---

### &#128721; STOP (3 Point)
**Pause to commit your changes to your Git repository!**

Take a moment to save your notebook, commit the changes to your Git repository with a meaningful commit message.

---

### Congratulations, you're done!

&#169; Copyright 2022,  Department of Computational Mathematics, Science and Engineering at Michigan State University